<a href="https://colab.research.google.com/github/yiiyama/qc-workbook-lecturenotes/blob/branch-2024/2024_06_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ToffoliゲートとShorコード

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\braket}[2]{\langle #1 | #2 \rangle}$

量子プログラミング演習として、Toffoliゲートの基本ゲートでの分解を考え、それを使ってShorコードでエラー訂正をしてみましょう。

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

!git clone -b branch-2024 https://github.com/UTokyo-ICEPP/qc-workbook-lecturenotes
!cp -r qc-workbook-lecturenotes/qc_workbook /root/.local/lib/python3.10/site-packages/

runtime_config_path = '/content/gdrive/MyDrive/qiskit-ibm.json'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Math
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.quantum_info.operators import Operator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

from qc_workbook.show_state import show_state, statevector_expr
from qc_workbook.utils import find_best_chain

In [ ]:
def show_circuit_op(circuit, global_phase=0.):
    """Compiles the LaTeX expression of the operation of the circuit on computational basis states."""
    op = Operator(circuit)

    unitary = op.data * np.exp(-1.j * global_phase)

    ket_template = fr'|{{:0{circuit.num_qubits}b}}\rangle'

    exprs = []
    for icol, col in enumerate(unitary.T):
        expr = statevector_expr(col, binary=True, state_label=None)
        exprs.append(fr'{ket_template.format(icol)} \rightarrow {expr}')

    return Math(r' \\ '.join(exprs))

## CPゲート

最初の問題は、制御$P$（$CP$）ゲートをCXと$R_z$から作ることです。おさらいをすると、CPゲートとは、2量子ビットゲートで、パラメータ$\phi$を取り、二つの量子ビットがともに$\ket{1}$であるような計算基底の位相を$\phi$前進させるゲートのことでした。

$$
\begin{align}
    CP(\phi) \ket{00} & = \ket{00} \\
    CP(\phi) \ket{01} & = \ket{01} \\
    CP(\phi) \ket{10} & = \ket{10} \\
    CP(\phi) \ket{11} & = e^{i\phi} \ket{11}
\end{align}
$$

**ヒント1**

まず制御$R_z$（$CR_z$）ゲートをCXと$R_z$から作ることを考えてみましょう。$CR_z$の作用は、左を制御ビット、右を標的ビットとして

$$
\begin{align}
    CR_z(\phi) \ket{00} & = \ket{00} \\
    CR_z(\phi) \ket{01} & = \ket{01} \\
    CR_z(\phi) \ket{10} & = e^{-i\phi/2} \ket{10} \\
    CR_z(\phi) \ket{11} & = e^{i\phi/2} \ket{11}
\end{align}
$$

です。$CR_z$が作れれば、制御ビットに$R_z$をかけることで、$\ket{00}, \ket{01}, \ket{10}$の位相を揃えることができ、$CP$ゲートに全体位相がかかった状態が実現できます。全体位相は無視していいので、それで$CP$ゲートの完成です。

**ヒント2**

$R_z$を$X$で挟むと、$R_z$のパラメータの符号を反転させたのと同じ作用が得られます。

In [ ]:
fig, axs = plt.subplots(1, 3, width_ratios=(8, 0.3, 4), figsize=(8, 2))
phi = Parameter('$\phi$')
circuit = QuantumCircuit(1)
circuit.rz(phi / 2., 0)
circuit.x(0)
circuit.rz(-phi / 2., 0)
circuit.x(0)
circuit.draw('mpl', ax=axs[0])
axs[1].text(0.5, 0.4, '=', fontsize=24)
axs[1].set_axis_off()
circuit = QuantumCircuit(1)
circuit.rz(phi, 0)
circuit.draw('mpl', ax=axs[2])

In [ ]:
cp_circuit = QuantumCircuit(2, name='CP')

phi = Parameter('$\phi$')

# cp_circuit が CP(phi)を実装するようにゲートを加えてください。
# phiは普通の変数のように使えます。
# 例： cp_circuit.rz(phi, 1)

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

cp_circuit.draw('mpl')

In [ ]:
# phiに具体的な値を入れて、CPが実装されていることを確認
phi_value = np.pi / 4.
circuit = cp_circuit.assign_parameters({phi: phi_value})
show_circuit_op(circuit, global_phase=(-phi_value / 4.))

$CP$は`QuantumCircuit`オブジェクトにもメソッド`cp`として備わっているので、以下では（回路図の見やすさを考慮して）標準メソッドを使うことにします。

## CCZゲート

次に、$CP$ゲートの特殊ケース（$\phi=\pi$）である制御$Z$（CZ）ゲートを二重制御に拡張したCCZゲートを、$CP$とCXを組み合わせて実装します。

$$
\begin{align}
    \text{CCZ} \ket{k} & = \ket{k} \; \text{for}\; 000 \leq k \leq 110 \\
    \text{CCZ} \ket{111} & = -\ket{111}
\end{align}
$$

In [ ]:
ccz_circuit = QuantumCircuit(3, name='CCZ')

# ccz_circuit が CCZを実装するようにゲートを加えてください。

ccz_circuit.cp(np.pi / 2., 1, 0)
ccz_circuit.cp(np.pi / 2., 2, 0)

# 上の二行で|111>の符号が反転しますが、同時に|011>と|101>に位相が乗ってしまうので、それを訂正する方法を考えてください。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

ccz_circuit.draw('mpl')

In [ ]:
show_circuit_op(ccz_circuit)

## Hadamardゲート

次にHadamardゲートの$R_z$と$\sqrt{X}$への分解を考えます。$\sqrt{X}$の作用は

$$
\begin{align}
    \sqrt{X} \ket{0} & = \frac{1}{\sqrt{2}} \left[e^{\frac{\pi}{4}i} \ket{0} + e^{-\frac{\pi}{4}i} \ket{1}\right] \\
    \sqrt{X} \ket{1} & = \frac{1}{\sqrt{2}} \left[e^{-\frac{\pi}{4}i} \ket{0} + e^{\frac{\pi}{4}i} \ket{1}\right]
\end{align}
$$

です。

**ヒント**

$R_z(\phi)\ket{0}$や$R_z(\phi)\ket{1}$に$\sqrt{X}$をかけるとどうなるか、また$\sqrt{X}$をかけた後の状態に$R_z(\phi)$をかけたらどうなるか、計算してみましょう。今回も全体位相が生じます。

In [ ]:
h_circuit = QuantumCircuit(1, name='H')

# h_circuit が Hを実装するようにゲートを加えてください。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

h_circuit.draw('mpl')

In [ ]:
show_circuit_op(h_circuit, global_phase=(-1. / 4. * np.pi))

## Toffoliゲート

$Z$ゲートを$H$ゲートで挟むと$X$ゲートと等価になることを思い出して、CCZと$H$からCCXを作ってください。

In [ ]:
toffoli_circuit = QuantumCircuit(3, name='Toffoli')

# toffoli_circuit が Toffoliを実装するようにゲートを加えてください。
# 量子ビット0が標的、1と2が制御とします。
# Hadamardゲートには toffoli_circuit.h()を使って構いません。

##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

toffoli_circuit.draw('mpl')

In [ ]:
show_circuit_op(toffoli_circuit)

## 量子エラーとエラー訂正

量子エラーは有限個の量子演算子の作用として捉えることができます。特に、1量子ビットのエラーは、量子ビットに$X$、$Z$、$XZ$のどれかがかかることと等価です。$X$がかかるようなエラーをビット反転（bit flip）エラー、$Z$のケースを位相反転（phase flip）エラー、$XZ$のケースを複合（combined bit-phase flip）エラーと呼びます。

### ビット反転（$X$）エラー

Shorコードの元になるのはビット反転コードという、量子ビットを3つ使うエラー訂正コードです。ビット反転コードは、その名が示唆するようにビット反転エラーに対してのみ耐性を持ちます。論理量子ビットでの計算途中で3つの量子ビットのうちのどれかにビット反転エラーが一度だけ起きるとき、デコーディングの過程でエラーが補正されます。

ビット反転コードのエンコーディングは

$$
\begin{align}
    \ket{0_L} & = \ket{000} \\
    \ket{1_L} & = \ket{111}
\end{align}
$$

です。ここで$\ket{0_L}$と$\ket{1_L}$はそれぞれ「論理$\ket{0}$状態」と「論理$\ket{1}$状態」を表します。

以下では、適当な状態に初期化した量子ビットをビット反転コードでエンコードし、論理量子ビットに論理$Z$ゲートをかける過程でどれかひとつの（物理）量子ビットにビット反転エラーが起きるというシナリオを考えます。デコーディングの際にToffoliゲートを使うと、エラーが補正されます。

In [ ]:
bitflip_circuit = QuantumCircuit(3)

# データ量子ビット（第0ビット）を適当な状態に初期化
theta = 1.2
phi = 0.7
lam = 0.
bitflip_circuit.u(theta, phi, lam, 0)
bitflip_circuit.barrier()

# データビットの|0>が|000>に、|1>が|111>にエンコードされるように量子ビットをエンタングルさせてください
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

bitflip_circuit.barrier()

# 論理ZゲートはZZZ
bitflip_circuit.z([0, 1, 2])

# 0から2のどれかひとつの量子ビットにX（ビット反転）がかかる
bitflip_circuit.x(np.random.randint(3))

bitflip_circuit.barrier()

# デコーディング：状態が|000>, |001>, |010>, |100>のときデータビットが|0>になり、
# |111>, |110>, |101>, |011>のとき|1>になるようにCXとToffoliを組んでください（Toffoliはすでに書き込んであります）
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################
# 0が標的、1と2が制御
bitflip_circuit.ccx(1, 2, 0)

bitflip_circuit.draw('mpl')

In [ ]:
show_state(bitflip_circuit, binary=True);

第1と第2量子ビット（Qiskitの順番なので左二桁）の状態が同一である＝第1と第2量子ビットがくくり出せる＝第0量子ビットとのエンタングルメントが切れていれば、正しくデコードされています。第0ビットの状態をエラーのない単一量子ビット回路のものと比較します。

In [ ]:
ref_circuit = QuantumCircuit(1)

ref_circuit.u(theta, phi, lam, 0)
ref_circuit.z(0)
show_state(ref_circuit, binary=True);

### ビット反転コードを実機で実行してみる

In [ ]:
service = QiskitRuntimeService(filename=runtime_config_path)
# 今一番空いているバックエンド
backend = service.least_busy(simulator=False, operational=True)

# 選ばれたバックエンドから一番性能の良い3量子ビットの並びを探す
best_chain = find_best_chain(backend, 3)

print(f'The job will run on {backend.name} qubits {best_chain}')

# 最もSWAPが少なくなる量子ビット対応（回路中のq0がbest_chain[1]にマップされる）
layout = [best_chain[1], best_chain[0], best_chain[2]]

bitflip_circuit_tr = transpile(bitflip_circuit.measure_all(inplace=False), backend=backend, initial_layout=layout)
ref_circuit_tr = transpile(ref_circuit.measure_all(inplace=False), backend=backend, initial_layout=[best_chain[1]])

bitflip_circuit_tr.draw('mpl', idle_wires=False)

In [ ]:
sampler = Sampler(backend)
shots = 2000
job = sampler.run([bitflip_circuit_tr, ref_circuit_tr], shots=shots)

In [ ]:
if job.status() == 'DONE':
    result = job.result()
    counts_bitflip = result[0].data.meas.get_counts()
    counts_ref = result[1].data.meas.get_counts()

    # bitflip_circuitの結果のq0のカウントとref_circuitのq0のカウントを理論値と比較する
    arr_bitflip = np.zeros(2)
    arr_ref = np.zeros(2)
    for ibit, bit in enumerate(['0', '1']):
        arr_bitflip[ibit] += sum(value for key, value in counts_bitflip.items() if key[-1] == bit)
        arr_ref[ibit] = counts_ref.get(bit, 0)

    arr_ideal = np.square(np.array([np.cos(theta / 2.), np.sin(theta / 2.)])) * shots

    plt.bar(np.arange(2) + 0.25, arr_bitflip, width=0.25, label='bitflip')
    plt.bar(np.arange(2) + 0.5, arr_ref, width=0.25, label='ref')
    plt.bar(np.arange(2) + 0.75, arr_ideal, width=0.25, label='ideal')
    plt.xticks(np.arange(2) + 0.5, ['0', '1'])
    plt.legend();

else:
    print('Job is not yet done, check again later!')

### 位相反転（$Z$）エラー

次に、位相反転を補正する3量子ビットのエラー訂正コードを考えます。$Z$を$H$で挟むと$X$になることを思い出すと、ビット反転コードを転用できることがすぐにわかります。

In [ ]:
phaseflip_circuit = QuantumCircuit(3)

phaseflip_circuit.u(theta, phi, lam, 0)
phaseflip_circuit.barrier()

# エンコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

phaseflip_circuit.barrier()

# 位相反転コードの論理ZゲートはXXX
phaseflip_circuit.x([0, 1, 2])

# ランダムに位相反転エラーが発生
phaseflip_circuit.z(np.random.randint(3))

phaseflip_circuit.barrier()

# デコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

phaseflip_circuit.ccx(1, 2, 0)

phaseflip_circuit.draw('mpl')

In [ ]:
show_state(phaseflip_circuit, binary=True);

### Shorコード

Shorコードは位相反転コードの物理量子ビットにビット反転コードの論理量子ビットを使って作ります。全部で9つの量子ビットを使い、全ての1量子ビットエラー（$X, Z, XZ$）を補正します。

In [ ]:
shor_circuit = QuantumCircuit(9)

shor_circuit.u(theta, phi, lam, 0)
shor_circuit.barrier()

# エンコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

shor_circuit.barrier()

# Shorコードの論理ZゲートもXXXXXXXXX
shor_circuit.x(range(9))

# ランダムにビットと位相が反転
erroneous_qubit = np.random.randint(9)
error = np.random.choice(['x', 'z', 'xz'])
if 'x' in error:
    shor_circuit.x(erroneous_qubit)
if 'z' in error:
    shor_circuit.z(erroneous_qubit)

shor_circuit.barrier()

# デコーディング
##################
### EDIT BELOW ###
##################

##################
### EDIT ABOVE ###
##################

shor_circuit.draw('mpl')

In [ ]:
if error == 'xz':
    global_phase = -np.pi
else:
    global_phase = 0.

show_state(shor_circuit, global_phase=global_phase, binary=True);